In [101]:
F = GF(2)
V = VectorSpace(F, 3)
G = GL(3, GF(2))

In [3]:
points = [U for U in V.subspaces(1)]
len(points)

7

In [4]:
planes = [U for U in V.subspaces(2)]
len(planes)

7

In [5]:
point_index = {points[i]: i for i in range(len(points))}
plane_index = {planes[i]: i for i in range(len(planes))}

In [7]:
edges = []
for p in points:
    for H in planes:
        if p.is_subspace(H):
            edges.append((p, H))
len(edges)

21

In [8]:
edge_index = {edges[i]: i for i in range(len(edges))}

In [9]:
num_vertices = len(points) + len(planes)
num_edges = len(edges)

boundary = Matrix(CDF, num_vertices, num_edges)

In [10]:
for (p, H), j in edge_index.items():
    i_p = point_index[p]
    i_H = plane_index[H] + len(points)

    boundary[i_H, j] = 1     # +H
    boundary[i_p, j] = -1    # -p

In [12]:
ker = boundary.right_kernel()
ker.dimension()


8

In [13]:
ker

Vector space of degree 21 and dimension 8 over Complex Double Field
Basis matrix:
[ 1.0  0.0 -1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 -1.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0 -1.0]
[ 0.0  1.0 -1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 -1.0  0.0  1.0  1.0  0.0 -1.0]
[ 0.0  0.0  0.0  1.0  0.0 -1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 -1.0  1.0  0.0  0.0  0.0  1.0  0.0 -1.0]
[ 0.0  0.0  0.0  0.0  1.0 -1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 -1.0  1.0  1.0  0.0 -1.0]
[ 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0 -1.0  0.0  0.0  0.0 -1.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0 -1.0]
[ 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0 -1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 -1.0  1.0  0.0  1.0 -1.0]
[ 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0 -1.0  0.0  0.0  0.0 -1.0  0.0  1.0  0.0  1.0 -1.0]
[ 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0 -1.0  0.0 -1.0  1.0  0.0  0.0  0.0  0.0  1.0 -1.0]

In [14]:
basis = ker.basis()
basis[0]

(1.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, -1.0)

In [27]:
def line_repr(L):
    """
    Canonical representative for a 1-dim subspace over F_2:
    return its unique nonzero vector.
    """
    for v in L.basis():
        if v != 0:
            return tuple(v)

In [45]:
def linear_form_str(a):
    """
    a is a vector in F_2^3, printed as a linear equation a·x = 0
    """
    terms = []
    for i, ai in enumerate(a):
        if ai != 0:
            terms.append(f"x{i+1}")
    if not terms:
        return "0"
    return " + ".join(terms)

In [49]:
def plane_repr(H):
    """
    Represent a plane as a readable equation like x1 + x2 = 0
    """
    V = H.ambient_vector_space()
    for a in V:
        if a != 0:
            if all(a * v == 0 for v in H.basis()):
                return "{" + linear_form_str(a) + " = 0}"

In [31]:
def edge_repr(edge):
    p, H = edge
    return f"{line_repr(p)} ⊂ {plane_repr(H)}"

In [47]:
def pretty_print_cycle(v, edges):
    for j, coeff in enumerate(v):
        if coeff != 0:
            print(f"{coeff:+} · [{edge_repr(edges[j])}]")


In [58]:
pretty_print_cycle(basis[0],edges)

+1.0 · [(1, 0, 0) ⊂ {x3 = 0}]
-1.0 · [(1, 0, 0) ⊂ {x2 = 0}]
-1.0 · [(0, 1, 0) ⊂ {x3 = 0}]
+1.0 · [(0, 1, 0) ⊂ {x1 = 0}]
+1.0 · [(0, 0, 1) ⊂ {x2 = 0}]
-1.0 · [(0, 0, 1) ⊂ {x1 = 0}]


In [89]:
from itertools import permutations

def apartment_cycle_from_g(g, edges, edge_index):
    V = g.base_ring()**3  # F^3
    basis = [g.matrix().column(i) for i in range(3)]
    lines = [V.subspace([v]) for v in basis]

    cycle = vector(CDF, len(edges))

    for w in permutations([0, 1, 2]):
        sign = Permutation([i+1 for i in w]).signature()
        p = lines[w[0]]
        H = lines[w[0]] + lines[w[1]]
        j = edge_index[(p, H)]
        cycle[j] += sign

    return cycle

In [111]:
g = G.random_element()

In [112]:
v_A = apartment_cycle_from_g(g, edges, edge_index); v_A

(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, -1.0, 0.0, -1.0, 1.0)

In [113]:
#verify this is actually a cycle by multiplying by boundary map
print((boundary * v_A).is_zero())

True


In [114]:
pretty_print_cycle(v_A,edges)

-1.0 · [(1, 1, 0) ⊂ {x1 + x2 + x3 = 0}]
+1.0 · [(1, 1, 0) ⊂ {x1 + x2 = 0}]
+1.0 · [(0, 1, 1) ⊂ {x1 + x2 + x3 = 0}]
-1.0 · [(0, 1, 1) ⊂ {x1 = 0}]
-1.0 · [(0, 0, 1) ⊂ {x1 + x2 = 0}]
+1.0 · [(0, 0, 1) ⊂ {x1 = 0}]
